In [77]:
import pandas as pd
import numpy as np
import random
import pickle

In [78]:
data_orig= pd.read_csv('../../MA_PREDICTOR/data/ma_data_car.csv', parse_dates=['announcement_date'])

In [79]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [80]:
data.head()

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,acquisition_count,bidder_count,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,15,1,0.006854
1,1653652020,2005-01-01,NaN,1680.57,Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,...,Friendly,Not Applicable,NaN,NaN,NaN,54201030,63103010,31,1,-0.010266
2,1949421020,2005-01-01,NaN,3301.76,Google Inc,GOOG,PhatBits,United States,United States,Private,...,Friendly,Not Applicable,NaN,NaN,NaN,57201030,57201020,2,1,0.007746
3,1652453040,2005-01-01,20.03,371.71,Graco Inc,GGG,Gusmer Europe SL,Spain,United States,Private,...,Friendly,Not Applicable,NaN,NaN,NaN,52102010,51101010,7,1,-0.011133
4,2721963020,2005-01-01,18220.52,NaN,Exxon Mobil Corp,XOM,Exxon Mobil Corp,United States,United States,Public,...,No Applicable,Open Market Purchase|Privately Negotiated Purc...,NaN,NaN,Other,50102030,50102030,8,1,-0.003971


In [81]:
data.columns

Index(['id', 'announcement_date', 'deal_value', 'acquiror_total_assets',
       'acquiror_name', 'acquiror_ticker', 'target_name', 'target_nation',
       'acquiror_nation', 'target_status', 'acquiror_industry',
       'target_industry', 'shares_at_announcement', 'shares_acquired',
       'consideration_offered', 'attitude', 'acquisition_technique',
       'acquiror_financial_advisor', 'target_financial_advisor', 'purpose',
       'acquiror_code', 'target_code', 'acquisition_count', 'bidder_count',
       'car'],
      dtype='object')

In [82]:
tmp = data[['consideration_offered', 'car', 'shares_acquired',
            'shares_at_announcement','acquiror_code', 'target_code', 'acquiror_nation',
            'target_nation', 'announcement_date', 'target_status',
            'acquisition_count', 'bidder_count']].copy()

# Consideration

In [83]:
# transform pipes into list of all considerations
list_gen = lambda x: x.split('|')
tmp['consideration_offered'] = tmp.consideration_offered.apply(list_gen)

# Cluster considerations
def cash(considerations):
    
    for consideration in considerations:
        if 'Cash' not in consideration:
            return 'Other'
    return 'Cash'

# Apply
tmp['consideration_offered'] = tmp.apply(lambda row: cash(row.consideration_offered), axis=1)

# Public_vs_others

In [84]:
tmp['target_status'].unique()

array(['Subsidiary', 'Private', 'Public', 'Joint Venture', 'Government'],
      dtype=object)

In [85]:
# Apply
tmp['target_status']=tmp['target_status'].apply(lambda val: "public" if val=='Public' else "others")

In [86]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count
0,Cash,0.006854,100.0,0.0,50102030,50103030,United States,United States,2005-01-01,others,15,1
1,Other,-0.010266,100.0,0.0,54201030,63103010,United States,United States,2005-01-01,others,31,1
2,Other,0.007746,100.0,0.0,57201030,57201020,United States,United States,2005-01-01,others,2,1
3,Cash,-0.011133,100.0,0.0,52102010,51101010,United States,Spain,2005-01-01,others,7,1
4,Cash,-0.003971,5.0,0.0,50102030,50102030,United States,United States,2005-01-01,public,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,100.0,0.0,52102010,59103010,United States,United States,2021-08-30,others,15,1
18455,Other,-0.040156,100.0,0.0,53203020,53205020,United States,United States,2021-08-30,others,60,1
18456,Other,-0.003545,100.0,0.0,54301020,57201010,United States,Norway,2021-08-30,others,52,1
18457,Other,-0.025992,100.0,0.0,55101010,52203030,United States,United States,2021-08-31,others,6,1


# Month

In [87]:
tmp['month']= pd.DatetimeIndex(tmp['announcement_date']).month

In [88]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month
0,Cash,0.006854,100.0,0.0,50102030,50103030,United States,United States,2005-01-01,others,15,1,1
1,Other,-0.010266,100.0,0.0,54201030,63103010,United States,United States,2005-01-01,others,31,1,1
2,Other,0.007746,100.0,0.0,57201030,57201020,United States,United States,2005-01-01,others,2,1,1
3,Cash,-0.011133,100.0,0.0,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1
4,Cash,-0.003971,5.0,0.0,50102030,50102030,United States,United States,2005-01-01,public,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,100.0,0.0,52102010,59103010,United States,United States,2021-08-30,others,15,1,8
18455,Other,-0.040156,100.0,0.0,53203020,53205020,United States,United States,2021-08-30,others,60,1,8
18456,Other,-0.003545,100.0,0.0,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8
18457,Other,-0.025992,100.0,0.0,55101010,52203030,United States,United States,2021-08-31,others,6,1,8


# Full vs not_full acqu

In [89]:
tmp['shares_acquired']=tmp['shares_acquired'].astype(str)

In [90]:
# Apply
tmp['shares_acquired']=tmp['shares_acquired'].apply(lambda val: "full" if val=="100.0" else "not_full")


In [91]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month
0,Cash,0.006854,full,0.0,50102030,50103030,United States,United States,2005-01-01,others,15,1,1
1,Other,-0.010266,full,0.0,54201030,63103010,United States,United States,2005-01-01,others,31,1,1
2,Other,0.007746,full,0.0,57201030,57201020,United States,United States,2005-01-01,others,2,1,1
3,Cash,-0.011133,full,0.0,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1
4,Cash,-0.003971,not_full,0.0,50102030,50102030,United States,United States,2005-01-01,public,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,0.0,52102010,59103010,United States,United States,2021-08-30,others,15,1,8
18455,Other,-0.040156,full,0.0,53203020,53205020,United States,United States,2021-08-30,others,60,1,8
18456,Other,-0.003545,full,0.0,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8
18457,Other,-0.025992,full,0.0,55101010,52203030,United States,United States,2021-08-31,others,6,1,8


# Shares at annoncement

In [92]:
tmp['shares_at_announcement']=tmp['shares_at_announcement'].astype(str)

In [93]:
# Apply
tmp['shares_at_announcement']=tmp['shares_at_announcement'].apply(lambda val: "no" if val=="0.0" else "yes")


In [94]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,others,15,1,1
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,others,31,1,1
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,others,2,1,1
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,public,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,others,15,1,8
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,others,60,1,8
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,others,6,1,8


# Cross-border vs National

In [95]:
tmp['cross_border'] = tmp.apply(lambda row: 'cross_border' if row.acquiror_nation == row.target_nation else 'national', axis=1)

In [96]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month,cross_border
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,others,15,1,1,cross_border
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,others,31,1,1,cross_border
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,others,2,1,1,cross_border
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1,national
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,public,8,1,1,cross_border
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,others,15,1,8,cross_border
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,others,60,1,8,cross_border
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8,national
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,others,6,1,8,cross_border


# Industry relatedness

In [97]:
# Transform codes

def relatedness(acquiror, target):
    if acquiror == target:
        return 'industry'
    elif acquiror[:6] == target[:6]:
        return 'industry_group'
    elif acquiror[:4] == target[:4]:
        return 'business_sector'
    elif acquiror[:2] == target[:2]:
        return 'economic_sector'
    else:
        return 'not_related'

In [98]:
tmp['relatedness'] = data.apply(lambda row: relatedness(str(row.acquiror_code), str(row.target_code)), axis=1)

In [99]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month,cross_border,relatedness
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,others,15,1,1,cross_border,business_sector
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,others,31,1,1,cross_border,not_related
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,others,2,1,1,cross_border,industry_group
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1,national,not_related
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,public,8,1,1,cross_border,industry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,others,15,1,8,cross_border,not_related
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,others,60,1,8,cross_border,business_sector
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8,national,not_related
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,others,6,1,8,cross_border,not_related


# Declassifier

In [100]:
# transform code into different cols mentioned above
def declassifier(x):
    x = str(x)
    return x[:2], x[:4]

In [101]:
tmp['economic_sector_ac'], tmp['business_sector_ac'] = \
zip(*data['acquiror_code'].map(declassifier))

In [102]:
tmp['economic_sector_target'], tmp['business_sector_target'] = \
zip(*data['target_code'].map(declassifier))

In [103]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,target_status,acquisition_count,bidder_count,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,others,15,1,1,cross_border,business_sector,50,5010,50,5010
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,others,31,1,1,cross_border,not_related,54,5420,63,6310
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,others,2,1,1,cross_border,industry_group,57,5720,57,5720
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,others,7,1,1,national,not_related,52,5210,51,5110
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,public,8,1,1,cross_border,industry,50,5010,50,5010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,others,15,1,8,cross_border,not_related,52,5210,59,5910
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,others,60,1,8,cross_border,business_sector,53,5320,53,5320
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,others,52,1,8,national,not_related,54,5430,57,5720
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,others,6,1,8,cross_border,not_related,55,5510,52,5220


# Columns modification

In [104]:
tmp.drop(['acquiror_nation', 'target_nation', 'announcement_date'], axis=1, inplace=True)

In [105]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,target_status,acquisition_count,bidder_count,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target
0,Cash,0.006854,full,no,50102030,50103030,others,15,1,1,cross_border,business_sector,50,5010,50,5010
1,Other,-0.010266,full,no,54201030,63103010,others,31,1,1,cross_border,not_related,54,5420,63,6310
2,Other,0.007746,full,no,57201030,57201020,others,2,1,1,cross_border,industry_group,57,5720,57,5720
3,Cash,-0.011133,full,no,52102010,51101010,others,7,1,1,national,not_related,52,5210,51,5110
4,Cash,-0.003971,not_full,no,50102030,50102030,public,8,1,1,cross_border,industry,50,5010,50,5010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,others,15,1,8,cross_border,not_related,52,5210,59,5910
18455,Other,-0.040156,full,no,53203020,53205020,others,60,1,8,cross_border,business_sector,53,5320,53,5320
18456,Other,-0.003545,full,no,54301020,57201010,others,52,1,8,national,not_related,54,5430,57,5720
18457,Other,-0.025992,full,no,55101010,52203030,others,6,1,8,cross_border,not_related,55,5510,52,5220


In [106]:
tmp= tmp.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1]]

# Implementation of the declassifier function

In [108]:
pickle_in = open("../../MA_PREDICTOR/data/declassification.pkl","rb")
results = pickle.load(pickle_in)

In [109]:
results.head()

,economic_sector,business_sector,industry_group,industry,activity,hierarchical_id
0,Energy,Energy - Fossil Fuels,Coal,Coal,Coal (NEC),50101010
1,Energy,Energy - Fossil Fuels,Coal,Coal,Coal Mining Support,50101010
2,Energy,Energy - Fossil Fuels,Coal,Coal,Coal Wholesale,50101010
3,Energy,Energy - Fossil Fuels,Oil & Gas,Integrated Oil & Gas,Integrated Oil & Gas,50102010
4,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC),50102020


In [110]:
def get_info_trbc(hierarchical_id):
    str_id= str(hierarchical_id)
    try:
        if len(str_id)==2:
            s = results.loc[results['hierarchical_id'].str.startswith(str_id), 'economic_sector'].reset_index(drop=True)
            return s[0]
        elif len(str_id)==4:
            r = results.loc[results['hierarchical_id'].str.startswith(str_id), 'business_sector'].reset_index(drop=True)
            return r[0]
        elif len(str_id)==6:
            q = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry_group'].reset_index(drop=True)
            return q[0] 
        elif len(str_id)==8:
            p = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry'].reset_index(drop=True)
            return p[0] 
        else:
            print("Please enter a valid hierarchical ID.")
    except:
        print("Please enter a valid hierarchical ID.")

In [111]:
tmp['economic_sector_ac'] = tmp.apply(lambda row: get_info_trbc(row.economic_sector_ac), axis=1)

In [112]:
tmp['business_sector_ac'] = tmp.apply(lambda row: get_info_trbc(row.business_sector_ac), axis=1)

In [113]:
tmp['economic_sector_target'] = tmp.apply(lambda row: get_info_trbc(row.economic_sector_target), axis=1)

In [114]:
tmp['business_sector_target'] = tmp.apply(lambda row: get_info_trbc(row.business_sector_target), axis=1)

# Uploading

In [116]:
tmp.to_csv('../../MA_PREDICTOR/data/ma_data_car_clean.csv', index=False, date_format="%d/%m/%Y")